In [2]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import random
import time
from datetime import datetime
from urllib.parse import urlparse, parse_qs

#### 0. 환경설정

In [3]:
delay = 1.5   # 딜레이 시간

#### 1. 초기화

In [4]:
driver = webdriver.Chrome('chromedriver')

In [5]:
# result 폴더 생성
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
# source 폴더 생성
if not any([s == 'source' for s in os.listdir('.')]):
    os.mkdir('source')

In [6]:
# 접속하기
#url = 'http://www.encar.com/ev/ev_carsearchlist.do?carType=ev&searchType=model&TG.R=D#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.A._.FuelType.%EC%A0%84%EA%B8%B0._.(C.GreenType.Y._.EvType.%EC%A0%84%EA%B8%B0%EC%B0%A8.))%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A%2250%22%7D'
url = 'http://www.encar.com/fc/fc_carsearchlist.do?carType=for&searchType=model&TG.R=B#!%7B%22action%22%3A%22(And.Hidden.N._.(C.CarType.N._.Manufacturer.%EB%B2%A4%EC%B8%A0.))%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A%2250%22%7D'
driver.get(url)
time.sleep(3*delay)

# [20개씩 보기] → [50개씩 보기]로 변환
viewer = Select(driver.find_element_by_css_selector('select#pagerow'))
viewer.select_by_value('50')
time.sleep(delay)

In [10]:
# 데이터 url 수집
batch = []
page = 123
while(True):
    try:
        driver.find_element_by_css_selector('div#pagination').find_element_by_xpath('//a[@data-page="{}"]'.format(page)).click()
    except:
        break
    time.sleep(delay)
    
    rows = driver.find_element_by_css_selector('tbody#sr_normal').find_elements_by_tag_name('tr')
    for row in rows:
        car_url = row.find_element_by_css_selector('td.inf > a').get_attribute('href')
        batch.append(car_url)
    print('현재페이지: p.{}'.format(page))
    page += 1
    

# url 저장
now = datetime.now().strftime('%Y%m%d%H%M%S')

# 결과 저장하기
df = pd.DataFrame(batch)
df[1] = df[0].apply(lambda x: parse_qs(urlparse(x).query)['carid'][0])
writer = pd.ExcelWriter('./result/car_url_{}.xlsx'.format(now), 'xlsxwriter')
df.to_excel(writer, index=False)
writer.save()
writer.close()

현재페이지: p.123


#### 2. page 저장하기

In [ ]:
# URL 데이터 불러오기
df = pd.read_excel('./result/car_url_20190903162242.xlsx')

# 데이터 저장하기
for i, url in df.iterrows():
    driver.get(url[0])
    time.sleep(2 + 3*random.random())
    with open('./source/car_{}.html'.format(i+1), 'w', -1, encoding='utf-8') as f:
        f.write(driver.page_source)

#### 3. page에서 데이터 추출하기

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
batch = []
for i in range(1, 50):
    with open('./source/car_{}.html'.format(i), 'r', encoding='utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html[html.find('<div id="areaBase"'):html.find('<div class="product_detail"')])
    result = {}
    try:
        # 일반
        result['브랜드'] = [soup.select('div.product_left > div.area_info > h1.prod_name > span.brand')[0].text]
        result['디테일'] = [soup.select('div.product_left > div.area_info > h1.prod_name > span.detail')[0].text]
        prod_infomain_list = soup.select('div.product_left > div.area_info > div.prod_infomain > ul > li')
        prod_infomain = {}
        for p in prod_infomain_list:
            key = p.select('span.blind')[0].text.replace(':', '')
            value = p.text.replace(' ', '').replace('\n', '').replace('자세히보기', '').replace(p.select('span.blind')[0].text, '')
            prod_infomain[key] = [value]
        result.update(prod_infomain)
        if not '수입형태' in result.keys():
            result['수입형태'] = ['']
        result['가격'] = [soup.select('div.product_left > div.area_info > div.prod_price > span.num')[0].text]
        result['등록번호'] = [soup.select('div.product_left > div.area_info2nd > div.prod_infoetc > ul.reg > li')[0].text.replace('등록번호', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')]
        result['조회수'] = [soup.select('div.product_left > div.area_info2nd > div.prod_infoetc > ul.reg > li')[1].text.replace('조회수', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')]
        result['찜'] = [soup.select('div.product_left > div.area_info2nd > div.prod_infoetc > ul.reg > li')[2].text.replace('찜', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')]
        #result['핫마크'] = [', '.join(list(map(lambda x: x.text.replace('\n', '').replace(' ', ''), soup.select('div.product_left > div.area_info2nd > div.prod_addon > span'))))]
        
        soup = BeautifulSoup(html[html.find('<div class="product_detail"'):html.find('<div><div id="ad_btm">')])
        option_classes = soup.select('div#areaOption > div.box_opt > div.con.option_hover')
        option_effective = []
        for option_class in option_classes:
            options = option_class.select('dl > dd.on')
            for option in options:
                try:
                    option_effective.append(option.select('a')[0].text.replace('\n', '').replace(' ', ''))
                except:
                    option_effective.append(option.select('p')[0].text.replace('\n', '').replace(' ', ''))
        df = pd.DataFrame.from_dict(result)
        df.insert(0, '진단구분', '일반')
        batch.append(df)
    except:
        # 진단
        result['브랜드'] = [soup.select('div#carPic > div.info_product > div.wrap_tit > strong.prod_name > span.brand')[0].text]
        result['디테일'] = [soup.select('div#carPic > div.info_product > div.wrap_tit > strong.prod_name > span.detail')[0].text]
        prod_infomain_list = soup.select('div#carPic > div.info_product > ul.list_carinfo')[0].select('li')
        prod_infomain = {}
        for p in prod_infomain_list:
            key = p.select('span.blind')[0].text.replace(':', '')
            value = p.text.replace(' ', '').replace('\n', '').replace('자세히보기', '').replace(p.select('span.blind')[0].text, '')
            prod_infomain[key] = [value]
        result.update(prod_infomain)
        if not '수입형태' in result.keys():
            result['수입형태'] = ['']
        result['가격'] = [soup.select('div#scrFix > div.wrap_keyinfo > div.info_purchase > em.emph_price')[0].text]
        result['등록번호'] = [soup.select('div#carPic > div.info_product > ul.list_carinfo')[1].select('li')[0].text.replace('등록번호', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')]
        result['조회수'] = [soup.select('div#carPic > div.info_product > ul.list_carinfo')[1].select('li')[1].text.replace('조회수', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')]
        result['찜'] = [soup.select('div#carPic > div.info_product > ul.list_carinfo')[1].select('li')[2].text.replace('찜', '').replace('\n', '').replace('자세히보기', '').replace(' ', '')]
        # result['핫마크'] = [', '.join(list(map(lambda x: x.text.replace('\n', '').replace(' ', ''), soup.select('div.product_left > div.area_info2nd > div.prod_addon > span'))))]

        soup = BeautifulSoup(html[html.find('<div class="product_detail"'):html.find('<div><div id="ad_btm">')])
        option_classes = soup.select('div#areaOption > div.area_options > dl.option_all')
        option_effective = []
        for option_class in option_classes:
            options = option_class.select('dd.on')
            for option in options:
                try:
                    option_effective.append(option.select('a')[0].text.replace('\n', '').replace(' ', ''))
                except:
                    option_effective.append(option.select('p')[0].text.replace('\n', '').replace(' ', ''))
        df = pd.DataFrame.from_dict(result)
        df.insert(0, '진단구분', '진단')
        batch.append(df)

In [6]:
now = datetime.now().strftime('%Y%m%d%H%M%S')
writer = pd.ExcelWriter('./result/result_{}.xlsx'.format(now), 'xlsxwriter')
df = pd.concat(batch)
df.to_excel(writer, index=False)
writer.save()
writer.close()